# Test alpha-spheres network

In [1]:
import molsysmt as msm

In [2]:
from scipy.spatial import Voronoi
from math import sqrt

In [3]:
from scipy.spatial.distance import euclidean

In [4]:
import numpy as np

In [5]:
from tqdm import tqdm

In [30]:
molecular_system = msm.convert('1HIV')
molecular_system = msm.extract(molecular_system, selection='molecule_type in ["protein", "peptide"]')

In [31]:
msm.info(molecular_system)

form,n_atoms,n_groups,n_components,n_chains,n_molecules,n_entities,n_proteins,n_structures
molsysmt.MolSys,1516,198,2,2,2,1,2,1


In [32]:
coordinates = msm.get(molecular_system, coordinates=True)

In [33]:
coordinates = msm.pyunitwizard.get_value(coordinates[0])
n_atoms = coordinates.shape[0]

In [34]:
radii = [0.2 for ii in range(n_atoms)]
probe = 0.1

In [35]:
voronoi = Voronoi(coordinates)

In [36]:
centers = voronoi.vertices
n_alphaspheres = centers.shape[0]

In [37]:
atoms_of_alphasphere = {ii:[] for ii in range(n_alphaspheres)}
atoms_of_alphasphere[-1] = []
alphaspheres_of_atom = {ii:[] for ii in range(n_atoms)}

In [38]:
for atom_index, region_index in enumerate(voronoi.point_region):
    for alphasphere_index in voronoi.regions[region_index]:
        atoms_of_alphasphere[alphasphere_index].append(atom_index)
        alphaspheres_of_atom[atom_index].append(alphasphere_index)

In [39]:
def atoms_are_permeable(atom_indices, coordinates, radii, probe_radius):

    centroid = coordinates[atom_indices,:].mean(axis=0)

    permeable=True
    
    for atom_index in atom_indices:
        distance = euclidean(coordinates[atom_index], centroid)
        if distance < radii[atom_index]+probe_radius:
            permeable=False
            break

    return permeable

In [41]:
edges=[]
permeability={}

for kk in tqdm(range(len(voronoi.ridge_vertices))):
    
    aux_ridge_vertices = np.sort(voronoi.ridge_vertices[kk])
    n = len(aux_ridge_vertices)

    for ii in range(n):
        aa = aux_ridge_vertices[ii]
        if aa!=-1:
            for jj in range(ii+1,n):
                bb = aux_ridge_vertices[jj]
                common = np.intersect1d(atoms_of_alphasphere[aa], atoms_of_alphasphere[bb], assume_unique=True)
                common = tuple(common)
                if len(common)>2:
                    if common not in permeability:
                        with_edge = atoms_are_permeable(common, coordinates, radii, probe)
                        permeability[common]=with_edge
                        if with_edge:
                            edges.append([aa,bb])

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11374/11374 [00:02<00:00, 4168.46it/s]


In [42]:
len(edges)

864

In [43]:
import networkx as nx

In [44]:
G = nx.Graph(edges)

In [45]:
G.number_of_nodes()

863

In [46]:
n_alphaspheres

9810

In [47]:
pockets = list(nx.connected_components(G))

In [48]:
len(pockets)

79

In [49]:
view=msm.view(molecular_system)
view

NGLWidget()

In [117]:
view.clear()
#view.add_surface(color='w', opacity=0.5)
view.add_ball_and_stick(color='w', opacity=0.5)

In [118]:
for ii in range(len(pockets)):
    if len(pockets[ii])>10:
        print(ii, len(pockets[ii]))

0 259
5 302
23 21
24 12
34 49
40 22


In [119]:
pocket = 40
alphaspheres_in_pocket = pockets[pocket]
atoms_in_pocket = np.concatenate([atoms_of_alphasphere[ii] for ii in alphaspheres_in_pocket])
atoms_in_pocket = np.unique(atoms_in_pocket)

print(alphaspheres_in_pocket)

{1285, 1286, 1287, 1295, 5392, 1298, 5403, 5404, 52, 733, 734, 735, 5216, 5214, 5218, 5219, 5215, 5221, 5222, 5223, 238, 4095}


In [120]:
sel='@'+','.join([str(ii) for ii in atoms_in_pocket])
view.add_surface(sel, color='red', opacity=0.3)

In [29]:
for ii in alphaspheres_in_pocket:
    aa=atoms_of_alphasphere[ii]
    view.shape.add_cylinder(coordinates[aa[0]]*10, coordinates[aa[1]]*10, [1,0,0], 0.1)
    view.shape.add_cylinder(coordinates[aa[0]]*10, coordinates[aa[2]]*10, [1,0,0], 0.1)
    view.shape.add_cylinder(coordinates[aa[0]]*10, coordinates[aa[3]]*10, [1,0,0], 0.1)
    view.shape.add_cylinder(coordinates[aa[1]]*10, coordinates[aa[2]]*10, [1,0,0], 0.1)
    view.shape.add_cylinder(coordinates[aa[1]]*10, coordinates[aa[3]]*10, [1,0,0], 0.1)
    view.shape.add_cylinder(coordinates[aa[2]]*10, coordinates[aa[3]]*10, [1,0,0], 0.1)